### 교차 검증 실습

In [1]:
import os, warnings
import numpy as np
# 경고 메세지 무시하거나 숨길때(ignore), 다시보이게(default)
# warnings.filterwarnings(action='ignore')
warnings.filterwarnings(action='ignore')

In [18]:
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd

In [3]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
train.shape, test.shape

((891, 12), (418, 11))

### 데이터 확인

In [4]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


### 결측치 처리

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


#### Age 결측치 평균으로 채우기

In [8]:
train.Age = train.Age.fillna(train.Age.mean())
test.Age = test.Age.fillna(train.Age.mean())
train.info(), test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass  

(None, None)

In [9]:
train.Embarked.value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [10]:
test.Embarked.value_counts()

S    270
C    102
Q     46
Name: Embarked, dtype: int64

In [12]:
train.Embarked = train.Embarked.fillna('S')
test.Embarked = test.Embarked.fillna('S')
train.info(), test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     891 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass  

(None, None)

### 데이터 형태 수치형으로 변환

In [13]:
train.Sex.value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [14]:
test.Sex.value_counts()

male      266
female    152
Name: Sex, dtype: int64

In [15]:
train['Sex_n'] = train.Sex.map({'male':1, 'female':2})
test['Sex_n'] = test.Sex.map({'male':1, 'female':2})
train.info(), test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     891 non-null    object 
 12  Sex_n        891 non-null    int64  
dtypes: float64(2), int64(6), object(5)
memory usage: 90.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Passenge

(None, None)

In [ ]:
train['Embarked_n'] = train['Embarked'].map({'S':1, 'C':2, 'Q':3})
test['Embarked_n'] = test['Embarked'].map({'S':1, 'C':2, 'Q':3})
train.info(), test.info()

### 데이터 분할 및 특성 선택

In [17]:
sel = ['Pclass', 'Age', 'Sex_n', 'Embarked_n']
X = train[sel]
y = train['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((668, 4), (668,), (223, 4), (223,))

### 모델 생성, 교차 검증

In [23]:
knn = KNeighborsClassifier()
knn.fit(X_train,  y_train)
# kfold = KFold(n_splits=cv 개수, shuffle=샘플을 섞는 여부)
kfold = KFold(n_splits=5, shuffle=True)
# scores = cross_val_score(모델 객체, 입력, 결과, cv = cross_validation 개수)
score = cross_val_score(knn, X, y, cv=kfold)
print(f"교차 검증 점수 : {score}")
print("교자 검증 점수 평균 :", score.mean())

교차 검증 점수 : [0.76536313 0.78651685 0.76966292 0.76966292 0.7752809 ]
교자 검증 점수 평균 : 0.7732973447994476


### knn 모델 test_size 조절하여 정확도 올리기

##### 훈련 : 테스트 = 9 : 1

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state=0)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((801, 4), (801,), (90, 4), (90,))

In [24]:
knn.fit(X_train,  y_train)
# kfold = KFold(n_splits=cv 개수, shuffle=샘플을 섞는 여부)
kfold = KFold(n_splits=5, shuffle=True)
# scores = cross_val_score(모델 객체, 입력, 결과, cv = cross_validation 개수)
score = cross_val_score(knn, X, y, cv=kfold)
print(f"교차 검증 점수 : {score}")
print("교자 검증 점수 평균 :", score.mean())

교차 검증 점수 : [0.81005587 0.74157303 0.74719101 0.78651685 0.75280899]
교자 검증 점수 평균 : 0.7676291507124475


##### 훈련 : 테스트 = 8 : 2

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((712, 4), (712,), (179, 4), (179,))

In [26]:
knn.fit(X_train,  y_train)
# kfold = KFold(n_splits=cv 개수, shuffle=샘플을 섞는 여부)
kfold = KFold(n_splits=5, shuffle=True)
# scores = cross_val_score(모델 객체, 입력, 결과, cv = cross_validation 개수)
score = cross_val_score(knn, X, y, cv=kfold)
print(f"교차 검증 점수 : {score}")
print("교자 검증 점수 평균 :", score.mean())

교차 검증 점수 : [0.7877095  0.78651685 0.76966292 0.74157303 0.76966292]
교자 검증 점수 평균 : 0.7710250455087566


##### 훈련 : 테스트 = 7 : 3

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=0)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((623, 4), (623,), (268, 4), (268,))

In [28]:
knn.fit(X_train,  y_train)
# kfold = KFold(n_splits=cv 개수, shuffle=샘플을 섞는 여부)
kfold = KFold(n_splits=5, shuffle=True)
# scores = cross_val_score(모델 객체, 입력, 결과, cv = cross_validation 개수)
score = cross_val_score(knn, X, y, cv=kfold)
print(f"교차 검증 점수 : {score}")
print("교자 검증 점수 평균 :", score.mean())

교차 검증 점수 : [0.73743017 0.76966292 0.78651685 0.78089888 0.78089888]
교자 검증 점수 평균 : 0.7710815391375306


##### 훈련 : 테스트 = 6 : 4

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state=0)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((534, 4), (534,), (357, 4), (357,))

In [30]:
knn.fit(X_train,  y_train)
# kfold = KFold(n_splits=cv 개수, shuffle=샘플을 섞는 여부)
kfold = KFold(n_splits=5, shuffle=True)
# scores = cross_val_score(모델 객체, 입력, 결과, cv = cross_validation 개수)
score = cross_val_score(knn, X, y, cv=kfold)
print(f"교차 검증 점수 : {score}")
print("교자 검증 점수 평균 :", score.mean())

교차 검증 점수 : [0.74860335 0.75280899 0.78089888 0.80898876 0.75842697]
교자 검증 점수 평균 : 0.7699453894921849


##### 훈련 : 테스트 = 5 : 5

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.5, random_state=0)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((445, 4), (445,), (446, 4), (446,))

In [32]:
knn.fit(X_train,  y_train)
# kfold = KFold(n_splits=cv 개수, shuffle=샘플을 섞는 여부)
kfold = KFold(n_splits=5, shuffle=True)
# scores = cross_val_score(모델 객체, 입력, 결과, cv = cross_validation 개수)
score = cross_val_score(knn, X, y, cv=kfold)
print(f"교차 검증 점수 : {score}")
print("교자 검증 점수 평균 :", score.mean())

교차 검증 점수 : [0.7877095  0.81460674 0.78651685 0.7247191  0.76404494]
교자 검증 점수 평균 : 0.7755194275312285


#### test_size 조절해서 교차 검증 점수 출력

In [33]:
for i in range(1, 6):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = (i/10), random_state=0)
    knn.fit(X_train,  y_train)
    # kfold = KFold(n_splits=cv 개수, shuffle=샘플을 섞는 여부)
    kfold = KFold(n_splits=5, shuffle=True)
    # scores = cross_val_score(모델 객체, 입력, 결과, cv = cross_validation 개수)
    score = cross_val_score(knn, X, y, cv=kfold)
    print(f"훈련 : 테스트 = {10-i} : {i}")
    print(f"교차 검증 점수 : {score}")
    print("교자 검증 점수 평균 :", score.mean())
    print()

훈련 : 테스트 = 9 : 1
교차 검증 점수 : [0.87709497 0.79775281 0.73595506 0.79213483 0.75842697]
교자 검증 점수 평균 : 0.7922729269976775

훈련 : 테스트 = 8 : 2
교차 검증 점수 : [0.73743017 0.78651685 0.83146067 0.75280899 0.79213483]
교자 검증 점수 평균 : 0.7800703031824744

훈련 : 테스트 = 7 : 3
교차 검증 점수 : [0.77094972 0.71348315 0.79775281 0.7752809  0.7752809 ]
교자 검증 점수 평균 : 0.766549494695876

훈련 : 테스트 = 6 : 4
교차 검증 점수 : [0.7877095  0.79775281 0.74157303 0.80337079 0.79213483]
교자 검증 점수 평균 : 0.7845081915761722

훈련 : 테스트 = 5 : 5
교차 검증 점수 : [0.76536313 0.80337079 0.78089888 0.79775281 0.74719101]
교자 검증 점수 평균 : 0.7789153223275376



## 데이터의 'Age'를 median 값으로 채워서 검증 하기

In [34]:
# 데이터 읽기
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
# 결측값 채우기(Age)
train.Age = train.Age.fillna(train.Age.median())
test.Age = test.Age.fillna(train.Age.median())
# 결측값 채우기(Embarked)
train.Embarked = train.Embarked.fillna('S')
test.Embarked = test.Embarked.fillna('S')
# 데이터 수치형으로 변환(Sex)
train['Sex_n'] = train.Sex.map({'male':1, 'female':2})
test['Sex_n'] = test.Sex.map({'male':1, 'female':2})
# 데이터 수치형으로 변환(Embarked)
train['Embarked_n'] = train['Embarked'].map({'S':1, 'C':2, 'Q':3})
test['Embarked_n'] = test['Embarked'].map({'S':1, 'C':2, 'Q':3})
# 데이터 특성 선택 및 분할
sel = ['Pclass', 'Age', 'Sex_n', 'Embarked_n']
X = train[sel]
y = train['Survived']
# 모델 생성
knn = KNeighborsClassifier()
# test_size 변경
for i in range(1, 6):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = (i/10), random_state=0)
    # 모델 훈련
    knn.fit(X_train,  y_train)
    # kfold = KFold(n_splits=cv 개수, shuffle=샘플을 섞는 여부)
    kfold = KFold(n_splits=5, shuffle=True)
    # scores = cross_val_score(모델 객체, 입력, 결과, cv = cross_validation 개수)
    score = cross_val_score(knn, X, y, cv=kfold)
    print(f"훈련 : 테스트 = {10-i} : {i}")
    print(f"교차 검증 점수 : {score}")
    print("교자 검증 점수 평균 :", score.mean())
    print()

훈련 : 테스트 = 9 : 1
교차 검증 점수 : [0.79329609 0.75842697 0.73033708 0.75280899 0.7752809 ]
교자 검증 점수 평균 : 0.7620300043939487

훈련 : 테스트 = 8 : 2
교차 검증 점수 : [0.79888268 0.75842697 0.75280899 0.7752809  0.78089888]
교자 검증 점수 평균 : 0.7732596823802649

훈련 : 테스트 = 7 : 3
교차 검증 점수 : [0.81564246 0.7752809  0.79213483 0.74719101 0.75842697]
교자 검증 점수 평균 : 0.7777352331931454

훈련 : 테스트 = 6 : 4
교차 검증 점수 : [0.7150838  0.71348315 0.7752809  0.80337079 0.82022472]
교자 검증 점수 평균 : 0.7654886698888959

훈련 : 테스트 = 5 : 5
교차 검증 점수 : [0.80446927 0.75842697 0.76404494 0.7247191  0.76404494]
교자 검증 점수 평균 : 0.7631410457598393

